In [1]:
import os

# import
import pandas as pd

In [2]:
# import files 
main_df = pd.read_csv('../PM2.5_Vehicles_Population.csv')
df_pop = pd.read_csv('../California_Population.csv') 

In [5]:
main_df
# main_df['Population Z Score'] = main_df['Population Z Score'].round(2)

,County,Data Year,Population_x,Z Score_x,Number of Gas Cars,Number of EV,Population_y,Z Score_y
0,Alameda,2010,1510271,0.033519,NaN,NaN,NaN,NaN
1,Amador,2010,38091,-0.251576,NaN,NaN,NaN,NaN
2,Butte,2010,220000,-0.216348,NaN,NaN,NaN,NaN
3,Calaveras,2010,45578,-0.250126,NaN,NaN,NaN,NaN
4,Colusa,2010,21419,-0.254805,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
865,Tulare,2024,478918,-0.166208,NaN,NaN,NaN,NaN
866,Tuolumne,2024,54407,-0.248416,NaN,NaN,NaN,NaN
867,Ventura,2024,823863,-0.099407,NaN,NaN,NaN,NaN
868,Yolo,2024,221666,-0.216026,NaN,NaN,NaN,NaN


In [ ]:
# total regestered car users/population 
def create_comparison(county):
    df = main_df[main_df['County'] == county]
    df.loc[:,'Total Cars'] = df['Number of Gas Cars'] + df['Number of EV']
    df.loc[:,'Total Cars/Population'] = (df['Total Cars'] / df['Population']).round(2)
    df.loc[:,'EV/Total Cars'] = (df['Number of EV'] / df['Total Cars']).round(2)
    df.loc[:,'Gas/Total Cars'] = (df['Number of Gas Cars'] / df['Total Cars']).round(2)
    return df[['Population Z Score','Total Cars/Population', 'EV/Total Cars', 'Gas/Total Cars']] 


# ev cars/population

In [ ]:
# so as more people got cars, more ev were regestered. 
# what is the relationship between population growth and ev adaptation?
## it seems to be positive, direct relationship 
## so as more population increases, ev car adaption also increases. i wonder if this is the same across counties? 

In [ ]:
san_diego = create_comparison('San Diego')
# san_diego

In [ ]:
los_angeles = create_comparison('Los Angeles')
# los_angeles

In [ ]:
orange = create_comparison('Orange')
# orange

In [ ]:
san_fran =  create_comparison('San Francisco')
# san_fran

In [ ]:
mono = create_comparison('Mono')
# mono 

In [ ]:
# is this crazy?
# function takes dataset from online and turns it into a merge-able df for main_df 

In [ ]:
def aged_population(dataset, datayear):
    # load files 
    df = pd.read_csv(dataset)
    df = df.loc[[28]]
    # extract unique county names
    county_names = [col.split("!!")[0] for col in df.columns if "California" in col]
    county_names = list(set(county_names))
    # extract county total and margin of error from main dataset
    county_total = [f"{county}!!Total!!Estimate" for county in county_names]
    county_margin_of_error = [f"{county}!!Total!!Margin of Error" for county in county_names]
    # making df of those
    county_total = df.loc[:, county_total]
    county_margin_of_error = df.loc[:, county_margin_of_error]
    # creating county total population estimate df 
    ct_df =county_total.transpose()
    ct_df = ct_df.reset_index() 
    ct_df.columns = ['County' , 'Total Population Estimate, >21']
    ct_df['County'] = ct_df['County'].str.split(' County').str[0]
    ct_df = ct_df.sort_values(by='County', ascending=True).reset_index(drop=True)
    
    # creating county margin of error 
    me_df = county_margin_of_error.transpose()
    me_df = me_df.reset_index() 
    me_df.columns = ['County' , 'Total Margin of Error, >21']
    me_df['County'] = me_df['County'].str.split(' County').str[0]
    me_df = me_df.sort_values(by='County', ascending=True).reset_index(drop=True) 
    
    # merging on county 
    ct_me_df = pd.merge(ct_df, me_df, how='inner', on = ['County'])
    ct_me_df['Data Year'] = datayear
    return ct_me_df

In [ ]:
testing = aged_population('/Users/clarissaclark/Desktop/isthiscrazy2022.csv', 2022)

testing

In [ ]:
aged_pop_df = pd.merge(main_df, testing, how='inner', on = ['Data Year', 'County'])
aged_pop_df